In [ ]:
import h5py as h5
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.models as models
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os
import csv
import h5py
from torchvision.models import resnet18
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from PIL import Image
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import warnings 
warnings.filterwarnings("ignore")

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.5 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
train_path = "/kaggle/input/deeplense-task1/dataset/train"
train_imgs = {}
for cat in os.listdir(train_path):
    sub_path = os.path.join(train_path,cat)
    imgs = []
    for img_path in os.listdir(sub_path):
        imgs.append(os.path.join(sub_path,img_path))
    train_imgs[cat] = imgs

In [3]:
val_path = "/kaggle/input/deeplense-task1/dataset/val"
val_imgs = {}
for cat in os.listdir(val_path):
    sub_path = os.path.join(val_path,cat)
    imgs = []
    for img_path in os.listdir(sub_path):
        imgs.append(os.path.join(sub_path,img_path))
    val_imgs[cat] = imgs

In [4]:
train_rows = [(img_path, label) for label, img_paths in train_imgs.items() for img_path in img_paths]
val_rows = [(img_path, label) for label, img_paths in val_imgs.items() for img_path in img_paths]

In [5]:
df_train = pd.DataFrame(train_rows, columns = ['img_path','label'])
df_val = pd.DataFrame(val_rows, columns = ['img_path','label'])

In [6]:
df_train['label'] = df_train['label'].map({'no':0, 'sphere':1, 'vort':2})
df_val['label'] = df_val['label'].map({'no':0, 'sphere':1, 'vort':2})

In [7]:
class Dataset(nn.Module):
    def __init__(self,data,transform = None, train = True):
        self.data = data
        self.imgs = [np.load(img_path) for img_path in self.data['img_path']]
        self.y = torch.tensor(self.data['label'], dtype = torch.long)
        self.transform = transform
        self.train = train

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.imgs[idx].squeeze()[:,:,np.newaxis]
        label = self.y[idx]
        img = np.tile(img,(1,1,3))
        if self.transform != None:
            img = self.transform(image = img)["image"]

        return img, label

In [8]:
train_transform = A.Compose([
    # A.RandomRotate90(p=0.5),
    # A.HorizontalFlip(p=0.5),
    # A.VerticalFlip(p=0.5),
    # A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=45, p=0.5),
    # A.Normalize(mean=(0.485, 0.456, 0.406), 
    #             std=(0.229, 0.224, 0.225)),
    # A.Resize(28,28),
    ToTensorV2()
])

val_transform = A.Compose([
    # A.Resize(28,28),
    ToTensorV2()
])

In [9]:
train_dataset = Dataset(df_train, train_transform)
val_dataset = Dataset(df_val, val_transform)

In [10]:
train_loader = DataLoader(train_dataset, batch_size = 32, shuffle = True, num_workers = 0)
val_loader = DataLoader(val_dataset, batch_size = 32, shuffle = False, num_workers = 0)

In [ ]:
class AuxPoissonCNN(nn.Module):
    def __init__(self, num_classes=3):
        super(AuxPoissonCNN, self).__init__()
        self.shared = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1), 
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 37 * 37, 256), 
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

        self.potential_head = nn.Sequential(
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 1, kernel_size=3, padding=1) 
        )
    
    def forward(self, x):
        features = self.shared(x)
        class_out = self.classifier(features)
        potential = self.potential_head(features)
        return class_out, potential

def poisson_loss(potential, source):
    source_gray = source.mean(dim=1, keepdim=True)
    laplace_kernel = torch.tensor([[0, 1, 0],
                                   [1, -4, 1],
                                   [0, 1, 0]], dtype=torch.float32, device=potential.device).unsqueeze(0).unsqueeze(0)
    laplacian_phi = F.conv2d(potential, laplace_kernel, padding=1)
    target = 2 * source_gray
    return F.mse_loss(laplacian_phi, target)

def get_pretrained_model(model_name, num_classes=3, pretrained=True):
    if model_name.lower() == 'auxpoissoncnn':
        model = AuxPoissonCNN(num_classes=num_classes)
        return model
    else:
        try:
            model = timm.create_model(model_name, pretrained=pretrained, num_classes=num_classes)
            return model
        except Exception as e:
            raise ValueError(f"Error creating model '{model_name}' with timm: {e}")

def train_aux_poisson(model_name, train_dataloader, val_dataloader, num_epochs=10, device='cuda', lambda_poisson=0.1):
    model = get_pretrained_model(model_name, num_classes=3, pretrained=True)
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()
    is_aux_model = hasattr(model, 'potential_head')
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        correct = 0
        total = 0
        
        epoch_true = []
        epoch_preds = []
        
        for images, labels in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]"):
            images = images.float().to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            
            if is_aux_model:
                outputs, potential = model(images)
                cls_loss = criterion(outputs, labels)
                aux_loss = poisson_loss(potential, images)
                loss = cls_loss + lambda_poisson * aux_loss
                preds = F.softmax(outputs, dim=1)
            else:
                outputs = model(images)
                loss = criterion(outputs, labels)
                preds = F.softmax(outputs, dim=1)
            
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item() * images.size(0)
            total += labels.size(0)
            _, predicted = (outputs.max(1) if is_aux_model else torch.max(outputs, 1))
            correct += predicted.eq(labels).sum().item()

            epoch_true.extend(labels.cpu().numpy())
            epoch_preds.extend(preds.cpu().detach().numpy())
   
            del images, labels, outputs, preds
            if is_aux_model:
                del potential, cls_loss, aux_loss
        
        try:
            auc = roc_auc_score(epoch_true, epoch_preds, multi_class='ovr')
        except Exception as e:
            auc = float('nan')
            print("AUC calculation error:", e)
        
        print(f"Epoch {epoch+1}: Train Loss {total_loss/total:.4f}, Train Acc {correct/total:.4f}, Train AUC {auc:.4f}")
        
        model.eval()
        val_loss = 0.0
        val_correct = 0
        total_val = 0
        val_true = []
        val_preds = []
        with torch.no_grad():
            for images, labels in tqdm(val_dataloader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]"):
                images = images.float().to(device)
                labels = labels.to(device)
                if is_aux_model:
                    outputs, potential = model(images)
                    cls_loss = criterion(outputs, labels)
                    aux_loss = poisson_loss(potential, images)
                    loss = cls_loss + lambda_poisson * aux_loss
                    preds = F.softmax(outputs, dim=1)
                else:
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    preds = F.softmax(outputs, dim=1)
                val_loss += loss.item() * images.size(0)
                total_val += labels.size(0)
                _, predicted = (outputs.max(1) if is_aux_model else torch.max(outputs, 1))
                val_correct += predicted.eq(labels).sum().item()
                val_true.extend(labels.cpu().numpy())
                val_preds.extend(preds.cpu().detach().numpy())
                
                del images, labels, outputs, preds
                if is_aux_model:
                    del potential, cls_loss, aux_loss
        
        try:
            val_auc = roc_auc_score(val_true, val_preds, multi_class='ovr')
        except Exception as e:
            val_auc = float('nan')
            print("Validation AUC calculation error:", e)
        
        print(f"Epoch {epoch+1}: Val Loss {val_loss/total_val:.4f}, Val Acc {val_correct/total_val:.4f}, Val AUC {val_auc:.4f}")
    return model

In [15]:
train_aux_poisson('efficientnet_b0', train_loader, val_loader)

model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

Epoch 1/10 [Train]: 100%|██████████| 938/938 [01:14<00:00, 12.64it/s]


Epoch 1: Train Loss 1.1217, Train Acc 0.4668, Train AUC 0.6568


Epoch 1/10 [Val]: 100%|██████████| 235/235 [00:06<00:00, 36.36it/s]


Epoch 1: Val Loss 0.9137, Val Acc 0.5800, Val AUC 0.8024


...

Epoch 9: Val Loss 0.2198, Val Acc 0.9203, Val AUC 0.9845


Epoch 10/10 [Train]: 100%|██████████| 938/938 [01:12<00:00, 12.86it/s]


Epoch 10: Train Loss 0.1563, Train Acc 0.9439, Train AUC 0.9923


Epoch 10/10 [Val]: 100%|██████████| 235/235 [00:05<00:00, 39.99it/s]

Epoch 10: Val Loss 0.2297, Val Acc 0.9203, Val AUC 0.9852


In [19]:
model = train_aux_poisson('efficientnet_b1', train_loader, val_loader)

Epoch 1/10 [Train]: 100%|██████████| 938/938 [01:39<00:00,  9.42it/s]


Epoch 1: Train Loss 1.1766, Train Acc 0.3516, Train AUC 0.5249


Epoch 1/10 [Val]: 100%|██████████| 235/235 [00:07<00:00, 30.52it/s]


Epoch 1: Val Loss 1.4142, Val Acc 0.3333, Val AUC 0.5676


Epoch 2/10 [Train]: 100%|██████████| 938/938 [01:39<00:00,  9.40it/s]


...

Epoch 9: Val Loss 0.2839, Val Acc 0.9008, Val AUC 0.9786


Epoch 10/10 [Train]: 100%|██████████| 938/938 [01:38<00:00,  9.55it/s]


Epoch 10: Train Loss 0.1979, Train Acc 0.9301, Train AUC 0.9871


Epoch 10/10 [Val]: 100%|██████████| 235/235 [00:07<00:00, 33.41it/s]

Epoch 10: Val Loss 0.4230, Val Acc 0.8865, Val AUC 0.9787


In [20]:
torch.save({'model_state_dict':model.state_dict()},'effnet_b1_poisson_loss.pth')

In [12]:
model = train_aux_poisson('resnet18', train_loader, val_loader)

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

Epoch 1/10 [Train]: 100%|██████████| 938/938 [00:45<00:00, 20.84it/s]


Epoch 1: Train Loss 0.9885, Train Acc 0.4526, Train AUC 0.6664


Epoch 1/10 [Val]: 100%|██████████| 235/235 [00:04<00:00, 51.78it/s]


Epoch 1: Val Loss 0.7437, Val Acc 0.6535, Val AUC 0.8388


...

Epoch 9: Val Loss 0.2763, Val Acc 0.9059, Val AUC 0.9811


Epoch 10/10 [Train]: 100%|██████████| 938/938 [00:44<00:00, 21.31it/s]


Epoch 10: Train Loss 0.1442, Train Acc 0.9471, Train AUC 0.9931


Epoch 10/10 [Val]: 100%|██████████| 235/235 [00:04<00:00, 52.54it/s]


Epoch 10: Val Loss 0.2479, Val Acc 0.9139, Val AUC 0.9829


In [13]:
torch.save({'model_state_dict':model.state_dict()},'resnet18_poisson_loss.pth')

In [17]:
model = train_aux_poisson('densenet121', train_loader, val_loader)

Epoch 1/10 [Train]: 100%|██████████| 938/938 [01:43<00:00,  9.06it/s]


Epoch 1: Train Loss 1.0799, Train Acc 0.3884, Train AUC 0.5751


Epoch 1/10 [Val]: 100%|██████████| 235/235 [00:09<00:00, 24.11it/s]


Epoch 1: Val Loss 0.8847, Val Acc 0.5620, Val AUC 0.7515


Epoch 2/10 [Train]: 100%|██████████| 938/938 [01:43<00:00,  9.07it/s]


...

Epoch 9: Val Loss 0.2231, Val Acc 0.9172, Val AUC 0.9846


Epoch 10/10 [Train]: 100%|██████████| 938/938 [01:43<00:00,  9.07it/s]


Epoch 10: Train Loss 0.1676, Train Acc 0.9398, Train AUC 0.9906


Epoch 10/10 [Val]: 100%|██████████| 235/235 [00:09<00:00, 24.27it/s]

Epoch 10: Val Loss 0.2411, Val Acc 0.9173, Val AUC 0.9826


In [18]:
torch.save({'model_state_dict':model.state_dict()},'densenet121_poisson_loss.pth')

In [19]:
model = train_aux_poisson('mobilenetv2_100', train_loader, val_loader)

model.safetensors:   0%|          | 0.00/14.2M [00:00<?, ?B/s]

Epoch 1/10 [Train]: 100%|██████████| 938/938 [00:52<00:00, 17.97it/s]


Epoch 1: Train Loss 1.1110, Train Acc 0.4471, Train AUC 0.6345


Epoch 1/10 [Val]: 100%|██████████| 235/235 [00:04<00:00, 51.97it/s]


Epoch 1: Val Loss 0.9622, Val Acc 0.5000, Val AUC 0.7298


...

Epoch 9: Val Loss 0.2742, Val Acc 0.8999, Val AUC 0.9770


Epoch 10/10 [Train]: 100%|██████████| 938/938 [00:52<00:00, 17.79it/s]


Epoch 10: Train Loss 0.2043, Train Acc 0.9245, Train AUC 0.9869


Epoch 10/10 [Val]: 100%|██████████| 235/235 [00:04<00:00, 51.23it/s]

Epoch 10: Val Loss 0.3103, Val Acc 0.8859, Val AUC 0.9729


In [20]:
torch.save({'model_state_dict':model.state_dict()},'mobilenetv2_poisson_loss.pth')